# H2O airlines demo

In [ ]:
import h2o

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (build 1.8.0_121-8u121-b13-0ubuntu1.16.04.2-b13); OpenJDK 64-Bit Server VM (build 25.121-b13, mixed mode)
  Starting server from /opt/conda/lib/python3.5/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp_vy6obz6
  JVM stdout: /tmp/tmp_vy6obz6/h2o_yonatan_started_from_python.out
  JVM stderr: /tmp/tmp_vy6obz6/h2o_yonatan_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster version:,3.10.4.8
H2O cluster version age:,3 days
H2O cluster name:,H2O_from_python_yonatan_uwrdre
H2O cluster total nodes:,1
H2O cluster free memory:,22.73 Gb
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [6]:
import h2o
import pandas
import pprint
import operator
import matplotlib
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from tabulate import tabulate

interactive = False
if not interactive: matplotlib.use('Agg', warn=False)
import matplotlib.pyplot as plt

In [4]:
print("Import and Parse airlines data")
data = h2o.import_file(path='https://s3.amazonaws.com/h2o-airlines-unpacked/allyears2k.csv')
data.describe()

Import and Parse airlines data
Parse progress: |█████████████████████████████████████████████████████████| 100%
Rows:43978
Cols:31




,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
type,int,int,int,int,int,int,int,int,enum,int,enum,int,int,int,int,int,enum,enum,int,int,int,int,enum,int,int,int,int,int,int,enum,enum
mins,1987.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,,1.0,,16.0,17.0,14.0,-63.0,-16.0,,,11.0,0.0,0.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,,
mean,1997.500000000007,1.4090909090909112,14.601073263904548,3.8206148528809805,1345.8466613820847,1313.222861430723,1504.6341303788763,1485.28916731093,,818.8429896766555,,124.81452913540394,125.02156260661943,114.3161110907823,9.3171119369843,10.00739065560008,,,730.1821905650545,5.3813680595305895,14.168634184732053,0.024694165264450407,,0.0024785119832643593,4.047800291055636,0.2893764692712416,4.855031904175528,0.017015560282100085,7.620060450016803,,
maxs,2008.0,10.0,31.0,7.0,2400.0,2359.0,2400.0,2359.0,,3949.0,,475.0,437.0,402.0,475.0,473.0,,,3365.0,128.0,254.0,1.0,,1.0,369.0,201.0,323.0,14.0,373.0,,
sigma,6.344360901707364,1.8747113713439594,9.175790425861416,1.9050131191329,465.3408991242314,476.2511399926006,484.3474879035155,492.75043412270236,,777.404369163635,,73.97444166059002,73.40159463000958,69.63632951506123,29.84022196241487,26.438809042916432,,,578.4380082304231,4.201979939864828,9.9050857472043,0.15519314135784237,,0.049723487218862286,16.205729904484198,4.416779898734125,18.619776221475668,0.4039401821015119,23.487565874106227,,
zeros,0,0,0,0,0,569,0,569,,0,,0,0,0,1514,6393,,,0,623,557,42892,,43869,7344,8840,7388,8914,7140,,
missing,0,0,0,0,1086,0,1195,0,0,0,32,1195,13,16649,1195,1086,0,0,35,16026,16024,0,9774,0,35045,35045,35045,35045,35045,0,0
0,1987.0,10.0,14.0,3.0,741.0,730.0,912.0,849.0,PS,1451.0,NA,91.0,79.0,nan,23.0,11.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,YES
1,1987.0,10.0,15.0,4.0,729.0,730.0,903.0,849.0,PS,1451.0,NA,94.0,79.0,nan,14.0,-1.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,NO
2,1987.0,10.0,17.0,6.0,741.0,730.0,918.0,849.0,PS,1451.0,NA,97.0,79.0,nan,29.0,11.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,YES


In [7]:
# ----------

# 2- Data exploration and munging. Generate scatter plots 
# of various columns and plot fitted GLM model.

# Function to fit a GLM model and plot the fitted (x,y) values
def scatter_plot(data, x, y, max_points = 1000, fit = True):
    if(fit):
        lr = H2OGeneralizedLinearEstimator(family = "gaussian")
        lr.train(x=x, y=y, training_frame=data)
        coeff = lr.coef()
    df = data[[x,y]]
    runif = df[y].runif()
    df_subset = df[runif < float(max_points)/data.nrow]
    df_py = h2o.as_list(df_subset)
    
    if(fit): h2o.remove(lr._id)

    # If x variable is string, generate box-and-whisker plot
    if(df_py[x].dtype == "object"):
        if interactive: df_py.boxplot(column = y, by = x)
    # Otherwise, generate a scatter plot
    else:
        if interactive: df_py.plot(x = x, y = y, kind = "scatter")
    
    if(fit):
        x_min = min(df_py[x])
        x_max = max(df_py[x])
        y_min = coeff["Intercept"] + coeff[x]*x_min
        y_max = coeff["Intercept"] + coeff[x]*x_max
        plt.plot([x_min, x_max], [y_min, y_max], "k-")
    if interactive: plt.show()

scatter_plot(data, "Distance", "AirTime", fit = True)
scatter_plot(data, "UniqueCarrier", "ArrDelay", max_points = 5000, fit = False)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [8]:
# Group flights by month
grouped = data.group_by("Month")
bpd = grouped.count().sum("Cancelled").frame
bpd.show()
bpd.describe()
bpd.dim

# Convert columns to factors
data["Year"]      = data["Year"]     .asfactor()
data["Month"]     = data["Month"]    .asfactor()
data["DayOfWeek"] = data["DayOfWeek"].asfactor()
data["Cancelled"] = data["Cancelled"].asfactor()

Month,nrow_Year,sum_Cancelled
1,41979,1067
10,1999,19


Rows:2
Cols:3




,Month,nrow_Year,sum_Cancelled
type,int,int,int
mins,1.0,1999.0,19.0
mean,5.5,21989.0,543.0
maxs,10.0,41979.0,1067.0
sigma,6.363961030678928,28270.12911183817,741.0479066835018
zeros,0,0,0
missing,0,0,0
0,1.0,41979.0,1067.0
1,10.0,1999.0,19.0


In [9]:
# Calculate and plot travel time
hour1 = data["CRSArrTime"] / 100
mins1 = data["CRSArrTime"] % 100
arrTime = hour1*60 + mins1

hour2 = data["CRSDepTime"] / 100
mins2 = data["CRSDepTime"] % 100
depTime = hour2*60 + mins2

# TODO: Replace this once list comprehension is supported. See PUBDEV-1286.
# data["TravelTime"] = [x if x > 0 else None for x in (arrTime - depTime)]
data["TravelTime"] = (arrTime-depTime > 0).ifelse((arrTime-depTime), h2o.H2OFrame([[None]] * data.nrow))
scatter_plot(data, "Distance", "TravelTime")

Parse progress: |█████████████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%


In [10]:
# Impute missing travel times and re-plot
data.impute(column = "Distance", by = ["Origin", "Dest"])
scatter_plot(data, "Distance", "TravelTime")

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [11]:
# ----------
# 3- Fit a model on train; using test as validation

# Create test/train split
s = data["Year"].runif()
train = data[s <= 0.75]
test  = data[s > 0.75]

# Set predictor and response variables
myY = "IsDepDelayed"
myX = ["Origin", "Dest", "Year", "UniqueCarrier", "DayOfWeek", "Month", "Distance", "FlightNum"]

# Simple GLM - Predict Delays
data_glm = H2OGeneralizedLinearEstimator(family="binomial", standardize=True)
data_glm.train(x               =myX,
               y               =myY,
               training_frame  =train,
               validation_frame=test)

# Simple GBM
data_gbm = H2OGradientBoostingEstimator(balance_classes=True,
                                        ntrees         =3,
                                        max_depth      =1,
                                        distribution   ="bernoulli",
                                        learn_rate     =0.1,
                                        min_rows       =2)

data_gbm.train(x               =myX,
               y               =myY,
               training_frame  =train,
               validation_frame=test)

# Complex GBM
data_gbm2 = H2OGradientBoostingEstimator(balance_classes=True,
                                         ntrees         =50,
                                         max_depth      =5,
                                         distribution   ="bernoulli",
                                         learn_rate     =0.1,
                                         min_rows       =2)

data_gbm2.train(x               =myX,
                y               =myY,
                training_frame  =train,
                validation_frame=test)

# Simple Random Forest
data_rf = H2ORandomForestEstimator(ntrees         =5,
                                   max_depth      =2,
                                   balance_classes=True)

data_rf.train(x               =myX,
              y               =myY,
              training_frame  =train,
              validation_frame=test)

# Complex Random Forest
data_rf2 = H2ORandomForestEstimator(ntrees         =10,
                                    max_depth      =5,
                                    balance_classes=True)

data_rf2.train(x               =myX,
               y               =myY,
               training_frame  =train,
               validation_frame=test)

# Deep Learning with 5 epochs
data_dl = H2ODeepLearningEstimator(hidden              =[10,10],
                                   epochs              =5,
                                   variable_importances=True,
                                   balance_classes     =True,
                                   loss                ="Automatic")

data_dl.train(x               =myX,
              y               =myY,
              training_frame  =train,
              validation_frame=test)

glm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [12]:
# Variable importances from each algorithm
# Calculate magnitude of normalized GLM coefficients
from six import iteritems
glm_varimp = data_glm.coef_norm()
for k,v in iteritems(glm_varimp):
    glm_varimp[k] = abs(glm_varimp[k])
    
# Sort in descending order by magnitude
glm_sorted = sorted(glm_varimp.items(), key = operator.itemgetter(1), reverse = True)
table = tabulate(glm_sorted, headers = ["Predictor", "Normalized Coefficient"], tablefmt = "orgtbl")
print("Variable Importances:\n\n" + table)

data_gbm.varimp()
data_rf.varimp()

Variable Importances:

| Predictor        |   Normalized Coefficient |
|------------------+--------------------------|
| Year.2008        |              2.22832     |
| Dest.LYH         |              1.61474     |
| Origin.LIH       |              1.55893     |
| Origin.MLB       |              1.50667     |
| Year.2007        |              1.48817     |
| Origin.MDW       |              1.44199     |
| Year.2003        |              1.41287     |
| Origin.CAE       |              1.32152     |
| Origin.LEX       |              1.32064     |
| Dest.HTS         |              1.32034     |
| Origin.TLH       |              1.2201      |
| Origin.HPN       |              1.20697     |
| Origin.HNL       |              1.18481     |
| Origin.LBB       |              1.17864     |
| Origin.IAH       |              1.1563      |
| Origin.BTV       |              1.11677     |
| Origin.GRR       |              1.05405     |
| UniqueCarrier.HP |              1.04573     |
| Year.1994      

| Dest.STT         |              0           |


[('Year', 926.7366943359375, 1.0, 0.48255442976355906),
 ('Origin', 424.71392822265625, 0.4582897502801368, 0.2211497491129153),
 ('FlightNum', 248.42453002929688, 0.26806376778606783, 0.12935535860427708),
 ('UniqueCarrier',
  215.3404541015625,
  0.23236422537025672,
  0.11212838627099535),
 ('Distance', 57.58279037475586, 0.06213500633641941, 0.029983522551025998),
 ('Dest', 47.682769775390625, 0.05145233815259489, 0.02482855369722724),
 ('Month', 0.0, 0.0, 0.0),
 ('DayOfWeek', 0.0, 0.0, 0.0)]

In [13]:
# Model performance of GBM model on test data
data_gbm2.model_performance(test)


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.20636957198600808
RMSE: 0.4542791784640895
LogLoss: 0.5997464768822931
Mean Per-Class Error: 0.31787387357578034
AUC: 0.7398580865248213
Gini: 0.47971617304964265
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3967952178967169: 


,NO,YES,Error,Rate
NO,2571.0,2703.0,0.5125,(2703.0/5274.0)
YES,1014.0,4831.0,0.1735,(1014.0/5845.0)
Total,3585.0,7534.0,0.3343,(3717.0/11119.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3967952,0.7221765,272.0
max f2,0.1311275,0.8480714,387.0
max f0point5,0.5630533,0.7038959,187.0
max accuracy,0.4942846,0.6832449,224.0
max precision,0.9695727,1.0,0.0
max recall,0.0838963,1.0,398.0
max specificity,0.9695727,1.0,0.0
max absolute_mcc,0.5630533,0.3650593,187.0
max min_per_class_accuracy,0.5161489,0.6808874,213.0
max mean_per_class_accuracy,0.5138797,0.6821261,214.0


Gains/Lift Table: Avg response rate: 52.57 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0102527,0.9267241,1.8689358,1.8689358,0.9824561,0.9824561,0.0191617,0.0191617,86.8935813,86.8935813
,2,0.0200558,0.9055457,1.6579763,1.7658211,0.8715596,0.9282511,0.0162532,0.0354149,65.7976315,76.5821080
,3,0.0302185,0.8913049,1.7002945,1.7437839,0.8938053,0.9166667,0.0172797,0.0526946,70.0294481,74.3783861
,4,0.0402914,0.8818846,1.6645210,1.7239681,0.875,0.90625,0.0167665,0.0694611,66.4520958,72.3968135
,5,0.0500045,0.8723484,1.6557140,1.7107101,0.8703704,0.8992806,0.0160821,0.0855432,65.5713969,71.0710132
,6,0.1000090,0.8164835,1.6012247,1.6559674,0.8417266,0.8705036,0.0800684,0.1656116,60.1224683,65.5967407
,7,0.1500135,0.7664114,1.4369965,1.5829771,0.7553957,0.8321343,0.0718563,0.2374679,43.6996511,58.2977108
,8,0.2000180,0.7268842,1.4575250,1.5516141,0.7661871,0.8156475,0.0728828,0.3103507,45.7525032,55.1614089
,9,0.3001169,0.6694558,1.3382825,1.4804610,0.7035040,0.7782439,0.1339607,0.4443114,33.8282542,48.0460954
,10,0.4000360,0.5928832,1.2208342,1.4156126,0.6417642,0.7441547,0.1219846,0.5662960,22.0834196,41.5612634
